from google.colab import drive
drive.mount('/content/drive')

In [376]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import mean_squared_error,mean_absolute_error


In [377]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings = pd.read_csv(Dataset_path+"ml-latest/ratings.csv")

In [378]:
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41890 non-null  object 
 10  language               41648 non-null  object 
 11  director               41886 non-null  object 
 12  writer                 41621 non-null  object 
 13  production_company     40944 non-null  object 
 14  actors                 41877 non-null  object 
 15  de

In [379]:
Movies_metadata['mean_vote'].describe()

count    41896.000000
mean         6.379342
std          1.036597
min          1.300000
25%          5.800000
50%          6.500000
75%          7.100000
max          9.400000
Name: mean_vote, dtype: float64

In [380]:
obj_Movies_metadata = Movies_metadata.select_dtypes(include=['object']).copy()
obj_Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   genres                 41896 non-null  object
 1   imdb_title_id          41896 non-null  object
 2   title                  41896 non-null  object
 3   original_title         41896 non-null  object
 4   date_published         41896 non-null  object
 5   genre                  41896 non-null  object
 6   country                41890 non-null  object
 7   language               41648 non-null  object
 8   director               41886 non-null  object
 9   writer                 41621 non-null  object
 10  production_company     40944 non-null  object
 11  actors                 41877 non-null  object
 12  description            41589 non-null  object
 13  budget                 15453 non-null  object
 14  usa_gross_income       13291 non-null  object
 15  worlwide_gross_inco

In [381]:
myobj = ['country','language','director','production_company']

for i in myobj:
    obj_Movies_metadata[i] = obj_Movies_metadata[i].astype('category')
    Movies_metadata[i] = obj_Movies_metadata[i].cat.codes


In [382]:
Movies_metadata = Movies_metadata.fillna(100000)
Movies_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41896 entries, 0 to 41895
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   movieId                41896 non-null  int64  
 1   genres                 41896 non-null  object 
 2   imdb_title_id          41896 non-null  object 
 3   title                  41896 non-null  object 
 4   original_title         41896 non-null  object 
 5   year                   41896 non-null  int64  
 6   date_published         41896 non-null  object 
 7   genre                  41896 non-null  object 
 8   duration               41896 non-null  int64  
 9   country                41896 non-null  int16  
 10  language               41896 non-null  int16  
 11  director               41896 non-null  int16  
 12  writer                 41896 non-null  object 
 13  production_company     41896 non-null  int16  
 14  actors                 41896 non-null  object 
 15  de

In [383]:
data =  Movies_metadata[['movieId','year','duration','metascore','reviews_from_users','reviews_from_critics'
                              ,'country','language','director','production_company','mean_vote']]

In [384]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(data.drop(['movieId','mean_vote'],axis=1))
data_norm = pd.DataFrame(scaled,columns=(data.drop(['movieId','mean_vote'],axis=1).columns))
data_norm= data_norm.join(data['mean_vote'])
data_norm= data_norm.join(data['movieId'])


from scipy import stats
z_scores = stats.zscore(data_norm.drop(['movieId','mean_vote'],axis=1))
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
newdata = data_norm[filtered_entries]

newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38725 entries, 0 to 41894
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  38725 non-null  float64
 1   duration              38725 non-null  float64
 2   metascore             38725 non-null  float64
 3   reviews_from_users    38725 non-null  float64
 4   reviews_from_critics  38725 non-null  float64
 5   country               38725 non-null  float64
 6   language              38725 non-null  float64
 7   director              38725 non-null  float64
 8   production_company    38725 non-null  float64
 9   mean_vote             38725 non-null  float64
 10  movieId               38725 non-null  int64  
dtypes: float64(10), int64(1)
memory usage: 3.5 MB


### Without Classing of mean_vote

In [385]:
target = newdata['mean_vote'] 
predictors = newdata.drop(['mean_vote','movieId'],axis=1)

In [386]:
X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)

In [389]:
model01 = Sequential()
model01.add(Dense(900, activation='relu', input_dim=9))
model01.add(Dense(600, activation='relu'))
model01.add(Dense(300, activation='relu'))
model01.add(Dense(150, activation='relu'))
model01.add(Dense(30, activation='relu'))
model01.add(Dense(1, activation='linear'))

# Compile the model
model01.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [390]:
model01.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.8012 - mean_squared_error: 1.0992
Epoch 2/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.7125 - mean_squared_error: 0.87050s - l
Epoch 3/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.6931 - mean_squared_error: 0.8322
Epoch 4/20
2711/2711 [==============================] - 29s 11ms/step - loss: 0.6868 - mean_squared_error: 0.8193
Epoch 5/20
2711/2711 [==============================] - 29s 11ms/step - loss: 0.6839 - mean_squared_error: 0.8139
Epoch 6/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.6813 - mean_squared_error: 0.8074
Epoch 7/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.6784 - mean_squared_error: 0.8022
Epoch 8/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6746 - mean_squared_error: 0.7989
Epoch 9/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.67

In [391]:
y_pred= model01.predict(X_test)

In [392]:
scores = model01.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1]))    

Accuracy on test data: 0.7407345771789551% 
 Error on test data: 0.2592654228210449


In [393]:
print('MSE:',mean_squared_error(y_test, y_pred))
print('MAE:',mean_absolute_error(y_test, y_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

MSE: 0.7407347214871505
MAE: 0.6559091103550981
RMSE: 0.8606594689464298


### With Classing mean_vote

In [463]:
newdata['mean_vote_class'] = 0
newdata['mean_vote_class'][newdata['mean_vote']>=7] = 1
newdata['mean_vote_class'][newdata['mean_vote']<7] = 0

<ipython-input-463-51378285559d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['mean_vote_class'] = 0
<ipython-input-463-51378285559d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['mean_vote_class'][newdata['mean_vote']>=7] = 1
C:\Users\PJ\anaconda3\lib\site-packages\pandas\core\generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inp

In [464]:
target = newdata['mean_vote_class'] 
predictors = newdata.drop(['mean_vote','mean_vote_class','movieId'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30, random_state=40)


In [473]:
model02 = Sequential()
model02.add(Dense(900, activation='relu', input_dim=9))
model02.add(Dense(600, activation='relu'))
model02.add(Dense(300, activation='relu'))
model02.add(Dense(150, activation='relu'))
model02.add(Dense(30, activation='relu'))
model02.add(Dense(1, activation='softmax'))

# Compile the model
model02.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_squared_error'])

In [475]:
model02.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 2/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 3/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 4/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 5/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 6/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 7/20
2711/2711 [==============================] - 28s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 8/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - mean_squared_error: 0.6896
Epoch 9/20
2711/2711 [==============================] - 27s 10ms/step - loss: 0.6896 - m

In [476]:
y_pred = model02.predict(X_test)

In [477]:
scores = model02.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores[1], 1 - scores[1])) 

Accuracy on test data: 0.6955586075782776% 
 Error on test data: 0.3044413924217224


In [478]:
print('MSE:',mean_squared_error(y_test, y_pred))
print('MAE:',mean_absolute_error(y_test, y_pred))
print('RMSE:',np.sqrt(mean_squared_error(y_test, y_pred)))

MSE: 0.6955586159407815
MAE: 0.6955586159407815
RMSE: 0.8340015683083465


### Generate Recommendation

In [426]:
import difflib
import random

def get_Movie_info(Movie_id):
    
    """
    Returns some basic information about a Movie given the Movie id and the metadata dataframe.
    """
    
    Movie_info = Movies_metadata[Movies_metadata['movieId'] == int(Movie_id)][['movieId', 'genres', 
                                                            'title', 'original_title', 'year', 'duration','mean_vote']]
    return Movie_info


def generate_recommendation(model,movieid):
    frames = pd.DataFrame()
    
    for i in movieid:
        mask = newdata[newdata.movieId.isin([i])]
        mask = mask.drop(['mean_vote','mean_vote_class','movieId'],axis=1)
        rating = model.predict(mask)
        tmp = get_Movie_info(i)
        tmp['rating'] = rating
        frames = frames.append(tmp, ignore_index=True)

    return frames
#.sort_values('rating', ascending=False)

In [432]:
generate_recommendation(model01,[7,8,34,54,45,200,3948,193882])

,movieId,genres,title,original_title,year,duration,mean_vote,rating
0,7,Comedy|Romance,Sabrina,Sabrina,1995,127,6.5,6.997986
1,8,Adventure|Children,Le avventure di Tom Sawyer e Huck Finn,Tom and Huck,1995,97,5.9,5.683178
2,34,Children|Drama,"Babe, maialino coraggioso",Babe,1995,91,7.0,6.752762
3,54,Children|Comedy,Una squadra di classe,The Big Green,1995,100,5.8,5.818998
4,45,Comedy|Drama|Thriller,Da morire,To Die For,1995,106,6.9,6.514626
5,200,Thriller,Legame mortale,The Tie That Binds,1995,99,5.2,5.914268
6,3948,Comedy,Ti presento i miei,Meet the Parents,2000,108,7.0,6.798370
7,193882,Adventure|Drama|Horror|Sci-Fi,Flora,Flora,2017,105,4.9,5.920873


In [479]:
generate_recommendation(model02,[7,8,34,54,45,200,3948,193882])

,movieId,genres,title,original_title,year,duration,mean_vote,rating
0,7,Comedy|Romance,Sabrina,Sabrina,1995,127,6.5,1.0
1,8,Adventure|Children,Le avventure di Tom Sawyer e Huck Finn,Tom and Huck,1995,97,5.9,1.0
2,34,Children|Drama,"Babe, maialino coraggioso",Babe,1995,91,7.0,1.0
3,54,Children|Comedy,Una squadra di classe,The Big Green,1995,100,5.8,1.0
4,45,Comedy|Drama|Thriller,Da morire,To Die For,1995,106,6.9,1.0
5,200,Thriller,Legame mortale,The Tie That Binds,1995,99,5.2,1.0
6,3948,Comedy,Ti presento i miei,Meet the Parents,2000,108,7.0,1.0
7,193882,Adventure|Drama|Horror|Sci-Fi,Flora,Flora,2017,105,4.9,1.0


In [487]:
newdata[newdata['mean_vote_class']==0]

,year,duration,metascore,reviews_from_users,reviews_from_critics,country,language,director,production_company,mean_vote,movieId,mean_vote_class
2,0.194197,0.014297,-1.616696,-0.169801,-0.211095,0.801757,-0.170536,-0.482289,0.211221,6.8,3,0
3,0.194197,1.101678,0.618633,-0.170530,-0.211095,0.801757,-0.776948,-0.767275,1.382431,6.2,4,0
4,0.194197,0.250684,-1.616629,-0.169497,-0.210648,0.801757,-0.776948,-1.247549,1.017691,6.2,5,0
6,0.194197,1.243510,-1.616472,-0.165424,-0.210201,-1.009939,-0.581186,1.353129,-0.811264,6.5,7,0
7,0.194197,-0.174812,0.618633,-0.172171,-0.211840,0.801757,-0.776948,0.785771,1.466493,5.9,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41887,-0.938584,-0.883973,0.618633,-0.173630,-0.211939,0.801757,-0.776948,-0.069591,1.373944,6.9,193855,0
41889,1.084240,-0.127535,0.618633,-0.172475,-0.212237,0.419483,-1.114469,-1.112799,-1.223289,6.3,193859,0
41892,-1.747714,-1.498579,0.618633,-0.173387,-0.212485,0.801757,-0.776948,-1.737879,-0.839554,6.7,193874,0
41893,0.841501,0.061575,0.618633,-0.173691,-0.212287,-1.577486,0.538258,0.830621,-1.214398,5.4,193878,0


In [488]:
y_pred.sum()

11618.0